<a href="https://colab.research.google.com/github/pyrated03/IISc-Incremental-Learning/blob/main/FSCIL_1st_100_Class_Training_CUB_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive
drive.mount('/content/gdrive')    

Mounted at /content/gdrive


# OLD STUFF


In [2]:
# !git clone https://github.com/xyutao/fscil.git

In [3]:
# cd /content/fscil/tools/

In [4]:
# !pip install -U mxnet-cu101==1.7.0

In [5]:
# import sys
# sys.path.append('/content/fscil/model')
# sys.path.append('//content/fscil/data')
# sys.path.append('/content/fscil/dataloader')
# sys.path.append('/content/fscil/params')
# sys.path.append('/content/fscil/results')
# sys.path.append('/content/fscil/script')
# sys.path.append('/content/fscil/tools')


In [6]:
# !pip install gluoncv

In [7]:
# !python3 "/content/gdrive/MyDrive/IISc Internship/FSCIL_Master/tools/train_cub200.py" --cuda

# NEW


In [8]:
!pip3 install resnet_pytorch

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import PIL
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image

import torchvision.transforms as T
import torch
import torch.nn as nn
import torchvision
from torchvision.utils import make_grid
from torchvision.utils import save_image    
from IPython.display import Image
import random


import torch.optim as optim
import torch.nn.functional as F
from torchvision import *
import time
import copy

In [10]:
import gc
import numpy as np
import pandas as pd
import cv2
import os
import time
import os.path as osp
import pickle
from PIL import Image
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler

class CUB2002011_ds(Dataset):
    def __init__(self, data_dir, mode="train"):
        assert mode in ["train", "test"] #Make sure the mode is one of the two
        super(CUB2002011_ds, self).__init__() #Inheritance dataset __init__()
        train_test_split = pd.read_csv(
            f"{data_dir}/train_test_split.txt",
            sep=" ",
            names=["img_id", "is_train"],
        )
        img_paths = pd.read_csv(
            f"{data_dir}/images.txt", sep=" ", names=["img_id", "img_path"]
        )  #Read all image paths
        labels = pd.read_csv(
            f"{data_dir}/image_class_labels.txt",
            sep=" ",
            names=["img_id", "img_label"],
        )
        self.data_dir = osp.join(data_dir, "images")
        self.mode = 1 if mode == "train" else 0  #Choose whether it is 1 training mode or test mode 0
        self.img_path = img_paths.loc[
            train_test_split["is_train"] == self.mode, "img_path"
        ].values  #Get the corresponding path in the corresponding mode
        self.label = labels.loc[
            train_test_split["is_train"] == self.mode, "img_label"
        ].values #Get the label in the corresponding mode
        img_sz = 84
        if mode == 'test':
            self.transform = transforms.Compose([
                transforms.Resize((img_sz, img_sz)),
                # transforms.ToPILImage(),
                transforms.ToTensor()
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((img_sz, img_sz)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(30),
                # transforms.ToPILImage(),
                transforms.ToTensor()
            ])
        self.y = self.label

    def __len__(self):
        return self.label.shape[0] #The size of the entire data set in the corresponding mode

    def __getitem__(self, idx):
        img = self.transform(
            Image.open(f"{self.data_dir}/{self.img_path[idx]}").convert("RGB")
        ) #Open the path of the image in the corresponding folder of the data set and convert the image data to RGB
        label = self.label[idx]
        return img, label-1
    

In [11]:
dset = CUB2002011_ds(data_dir="gdrive/My Drive/IITB Internship/CUB_200_2011", mode = "train")
batch_size = 32

dataset_size = len(dset)
indices = list(range(dataset_size))
split1 = int(np.floor(0.4 * dataset_size))
split2 = int(np.floor(0.5 * dataset_size))
train_sampler = indices[:split1]
val_sampler = indices[split1:split2]
train_ds = DataLoader(dset, batch_size=batch_size, sampler=train_sampler, drop_last=True)
val_ds = DataLoader(dset, batch_size=batch_size, sampler=val_sampler, drop_last=True)

In [12]:
test_dset = CUB2002011_ds(data_dir="gdrive/My Drive/IITB Internship/CUB_200_2011", mode = "test")
batch_size = 32

dataset_size_test = len(test_dset)
indices = list(range(dataset_size_test))
split = int(np.floor(0.5 * dataset_size_test))
test_sampler = indices[:split]
test_ds = DataLoader(test_dset, batch_size=batch_size, sampler=test_sampler, drop_last=True)

In [13]:
# dset[1].show()

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [15]:
dataset_sizes = {'train':len(train_ds),'val':len(val_ds), 'test':len(test_ds)}

### **DEFINING RESNET-18**

In [16]:
import torchvision.models as models

model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
model.fc = nn.Sequential(nn.Linear(512,93))#, nn.Softmax(93))
# model
model = model.to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
def train_model(model, criterion, softmax, optimizer, scheduler, device, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            if phase == 'train':
                for inputs, labels in train_ds:
                    inputs = inputs.to(device)
                    # print(inputs.size(0))
                    # print("IN", labels.shape)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        # print(outputs.shape)
                        _, preds = torch.max(outputs, 1)
                        # print(preds)
                        # pred_labels = softmax(outputs)
                        # one_hot = torch.zeros(outputs.shape)
                        # one_hot[torch.arange(32), labels] = int(1)
                        # one_hot = one_hot.type(torch.LongTensor)
                        # one_hot = one_hot.to(device)
                        # outputs = softmax(outputs)
                        # print(outputs)
                        # print(one_hot)
                        loss = criterion(outputs, labels)
                        print(loss)
                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
            elif phase == 'val':
                for inputs, labels in val_ds:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        # outputs = softmax(outputs)
                        # pred_labels = softmax(outputs)
                        loss = criterion(outputs, labels)
                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    # running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [19]:
criteria = nn.CrossEntropyLoss()
softmax = nn.Softmax()
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.0001)

scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
# Number of epochs
eps=5

In [ ]:
model = train_model(model, criteria, softmax, optimizer, scheduler, device, eps)

In [27]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
# output.backward()

In [28]:
input

tensor([[ 0.0845, -1.1671,  0.7568, -0.4110, -0.6324],
        [-0.5115, -0.1504, -0.8352, -0.2928, -1.3581],
        [-0.7348,  1.9715, -0.4863,  0.4978, -0.6152]], requires_grad=True)

In [29]:
target

tensor([2, 2, 3])

In [30]:
output

tensor(1.5150, grad_fn=<NllLossBackward>)